In [1]:
import requests, bs4
import time, os, requests, json
import pandas as pd
from bs4 import BeautifulSoup 

# 2017, 2022

In [2]:
def createDF(path):
    HTMLFile = open(path, "r") 
    S = BeautifulSoup(HTMLFile.read() , 'lxml') 
    text = S.find_all('tbody')[0].find_all('td')
    count = 1; rec = {1:[], 2:[], 3:[], 4:[], 5:[]}
    for i in text:
        info = i.find_all('a')
        if len(info)==2: 
            #print(info)
            for c in info:
                if 'export' not in c['href']: continue
                rec[count].append(c['href']); count += 1
        elif len(info)==1:
            content = info[0]['href']
            if 'export' not in content: continue
            if 'SU' in info[0]['alt']: 
                rec[count].append(content); count += 1
                rec[count].append(''); count += 1
            elif 'MU' in info[0]['alt']: 
                rec[count].append(''); count += 1
                rec[count].append(content);count += 1
        else: 
            if count == 6: count = 1
            content = i.text
            if len(content) < 1: continue
            rec[count].append(content) 
            count += 1
    rec_df = pd.DataFrame(rec)
    return rec_df.rename(columns={1: "Code", 2: 'Industry', 3:'Type', 4:'SU', 5:'MU'})

* 2022 surveys: manually download the html from https://bhs.econ.census.gov/ombpdfs2022/
* 2017 surveys: manually download the html from https://bhs.econ.census.gov/ombpdfs/

In [3]:
#rec_df = createDF("/Users/bjcliang/Desktop/Nate/Census/ES2022.html") #797 rows
rec_df = createDF("/Users/bjcliang/Desktop/Nate/Census/ES2017.html") #819 rows

In [ ]:
path = '/Users/bjcliang/Desktop/Nate/Census/Surveys2017'
#for i, row in rec_df[rec_df['Type']=='2022_ECON-SERVICES'].iterrows():
for i, row in rec_df[rec_df['Type']=='2017 ECON-SERVICES'].iterrows():
    if i%10==0: print(i, end = '\r')
    for k in ['SU', 'MU']:
        name = row['Code']
        if len(row[k]) == 0: continue
        pdf_response = requests.get(row[k])
        target = f'{path}/{name}{k}.pdf'
        with open(target, 'wb') as pdf_file: pdf_file.write(pdf_response.content)

In [30]:
rec_df[rec_df['Type']=='2017 ECON-SERVICES']

,Code,Industry,Type,SU,MU
0,AE-7110C,"Performing Arts, Spectator Sports, and Related...",2017 ECON-SERVICES,https://bhs.econ.census.gov/ombpdfs/export/AE-...,
1,AE-71110,Performing Arts Companies,2017 ECON-SERVICES,https://bhs.econ.census.gov/ombpdfs/export/AE-...,https://bhs.econ.census.gov/ombpdfs/export/AE-...
2,AE-71120,Spectator Sports,2017 ECON-SERVICES,https://bhs.econ.census.gov/ombpdfs/export/AE-...,https://bhs.econ.census.gov/ombpdfs/export/AE-...
3,AE-71130,"Promoters of Performing Arts, Sports, and Simi...",2017 ECON-SERVICES,https://bhs.econ.census.gov/ombpdfs/export/AE-...,https://bhs.econ.census.gov/ombpdfs/export/AE-...
4,AE-71140,"Agents and Managers for Artists, Athletes, Ent...",2017 ECON-SERVICES,https://bhs.econ.census.gov/ombpdfs/export/AE-...,https://bhs.econ.census.gov/ombpdfs/export/AE-...
...,...,...,...,...,...
726,UT-2218A,Other Electric Power Generation (Consolidated),2017 ECON-SERVICES,,https://bhs.econ.census.gov/ombpdfs/export/UT-...
727,UT-2221A,Electric Bulk Power Transmission and Control (...,2017 ECON-SERVICES,,https://bhs.econ.census.gov/ombpdfs/export/UT-...
728,UT-2222A,Electric Power Distribution (Consolidated),2017 ECON-SERVICES,,https://bhs.econ.census.gov/ombpdfs/export/UT-...
729,UT-2232A,Sewage Treatment Facilities (Consolidated),2017 ECON-SERVICES,,https://bhs.econ.census.gov/ombpdfs/export/UT-...


In [4]:
service_naics = set([i[3:5] for i in rec_df[rec_df['Type']=='2017 ECON-SERVICES']['Code']])
print(service_naics)

{'62', '61', '53', '72', '56', '81', '51', '22', '54', '71', '52', '49', '48', '55'}


# 2012

https://www2.census.gov/programs-surveys/economic-census/2012/questionnaires/forms/ has more draft surveys. Not important here

In [21]:
#HTMLFile = open('/Users/bjcliang/Desktop/Nate/Census/ES2012_w2.htm', "r") 
#S = BeautifulSoup(HTMLFile.read() , 'lxml') 
#href = []
#for i in S.find_all('a'):
#    try: 
#        if '.pdf' in i['href']:
#            href.append(i['href'])
#    except KeyError: pass
#set(href)-set(rec_df[(rec_df['class']=='forms')]['href'])

In [5]:
HTMLFile = open('/Users/bjcliang/Desktop/Nate/Census/ES2012.html', "r") 
S = BeautifulSoup(HTMLFile.read() , 'lxml') 
text = S.find_all('a', {'class':"uscb-layout-align-start-start uscb-text-decoration-some"})   

In [24]:
rec_df = pd.DataFrame({'Code': [i['name'].split('(')[-1].replace(')', '').replace(' ', '') for i in text],
                       'Industry': [i['name'].split('(')[0].strip() for i in text],
                       'href': [i['href'] for i in text]})
rec_df['Type'] = rec_df['Code'].apply(lambda x: x.split('-')[-1][:2].strip())
rec_df['class'] = rec_df['href'].apply(lambda x: x.split('/')[-2])
service_naics = {'62', '61', '53', '72', '56', '81', '51', '22', '54', '71', '52', '49', '48', '55'}
service = rec_df[(rec_df['class']=='forms')&(rec_df['Type'].isin(service_naics))]
service

,Code,Industry,href,Type,class
0,AF-72101,Traveler Accommodation,https://www2.census.gov/programs-surveys/econo...,72,forms
1,AF-72102,RV Parks and Recreational Camps,https://www2.census.gov/programs-surveys/econo...,72,forms
2,AF-72201,Food Services and Drinking Places,https://www2.census.gov/programs-surveys/econo...,72,forms
3,AF-72202,Special Food Services,https://www2.census.gov/programs-surveys/econo...,72,forms
4,AF-72290,Classification Form - Accommodation and Food S...,https://www2.census.gov/programs-surveys/econo...,72,forms
...,...,...,...,...,...
495,TW-48560,Transit and Ground Passenger Transportation,https://www2.census.gov/programs-surveys/econo...,48,forms
496,TW-48601,Pipelines,https://www2.census.gov/programs-surveys/econo...,48,forms
497,TW-48801,Transportation Services,https://www2.census.gov/programs-surveys/econo...,48,forms
498,UT-22101,"Electric, Gas, and Water Utilities",https://www2.census.gov/programs-surveys/econo...,22,forms


In [8]:
path = '/Users/bjcliang/Desktop/Nate/Census/Surveys2012'
for i, row in service.iterrows():
    if i%10==0: print(i, end = '\r')
    name = row['Code']
    pdf_response = requests.get(row['href'])
    target = f'{path}/{name}.pdf'
    with open(target, 'wb') as pdf_file: pdf_file.write(pdf_response.content)

# 2002, 2007
Surveys not found